In [2]:
## Required Libraries
import re
import nltk
import string
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('twitter_samples')
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
stemmer = PorterStemmer()
stopwords_english = stopwords.words('english')
punctuations = string.punctuation

[nltk_data] Downloading package punkt to C:\Users\Nikhil
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to C:\Users\Nikhil
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Nikhil
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
### Loading the Dataframe cell 

df_test = pd.read_csv("C:\\Users\\Nikhil Gupta\\Desktop\\assignment spring 2022\\Data Mining\\project\\Personality_perception-main\\data\\Tweets\\twitter_data.csv")

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
train_pos = all_positive_tweets[:]
train_neg = all_negative_tweets[:]
train_x = train_pos+train_neg
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
df_train = pd.DataFrame()
df_train["text"] = train_x
df_train["sentiment"] = train_y

In [4]:
class CleanData:
    
    def __init__(self, df_test,df_train):
         self.df_test_cleaned = df_test
         self.df_train_cleaned = df_train
         self.Openness_list = []
         self.Agreeableness_list = []
         self.Conscientiousness_list = []
         self.Extroversion_list = []
         self.Neuroticism_list = []
         self.all_positive_tweets = []
         self.all_negative_tweets = []
         self.stopwords_english = stopwords.words('english')
        
    def read_persona_dict(self, Personality):
        for i in Personality:
            path = f"C:\\Users\\Nikhil Gupta\\Desktop\\assignment spring 2022\\Data Mining\\project\\Personality_perception-main\\data\\Dictionary\\{i}.txt"
            with open(path) as file:
                if i == "Openness":
                   self.Openness_list = [line.rstrip().lower() for line in file]
                   self.Openness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Openness_list]
                elif i == "Agreeableness":
                   self.Agreeableness_list = [line.rstrip().lower() for line in file]
                   self.Agreeableness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Agreeableness_list]
                elif i == "Conscientiousness":
                   self.Conscientiousness_list = [line.rstrip().lower() for line in file]
                   self.Conscientiousness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Conscientiousness_list]
                elif i == "Extroversion":
                   self.Extroversion_list = [line.rstrip().lower() for line in file]
                   self.Extroversion_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Extroversion_list]
                elif i == "Neuroticism":
                   self.Neuroticism_list = [line.rstrip().lower() for line in file]
                   self.Neuroticism_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Neuroticism_list]


        
    
    def pre_processing_dataframe(self,df):
        stopwords_english = stopwords.words('english')
        punctuations = string.punctuation
        stemmer = PorterStemmer()
        #### Removing the unwanted things in the tweets
        df["text"]=df["text"].apply(lambda x: re.sub(r'RT[\s]+','', str(x)))
        df['text']=df['text'].apply(lambda x: re.sub(r"http\S+",'', str(x)))
        df['text']=df['text'].apply(lambda x: re.sub(r'#', '', str(x)))
        df['tokenized_tweets']=df['text'].apply(lambda x: word_tokenize(x))
        
        df['tokenized_tweets'] = df['tokenized_tweets'].apply(lambda x: [word.lower() for word in x if (word not in stopwords_english and word not in punctuations)])


        
    def pre_process(self, x):
        if x == "train":
            self.pre_processing_dataframe(self.df_train_cleaned)
        elif x == "test":
            self.pre_processing_dataframe(self.df_test_cleaned)
        
    def get_persona_scores(self):
        max_list = []
        persona_list = []
        
        for i in range(len(self.df_test_cleaned.index)):
            max_score = 0
            open_count, agree_count , extro_count , consci_count, neuro_count = 0 , 0 , 0 , 0 , 0
            open_ratio, agree_ratio , extro_ratio , consci_ratio, neuro_ratio = 0 , 0 , 0 , 0 , 0
        
            for j in range(len(self.df_test_cleaned["tokenized_tweets"][i])):

                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Openness_list):
                    open_count += 1
                    open_ratio = open_count/len(self.Openness_list)
                    
                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Agreeableness_list):
                    agree_count +=1
                    agree_ratio = agree_count/len(self.Agreeableness_list)
                    
                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Extroversion_list):
                    extro_count +=1
                    extro_ratio = extro_count/len(self.Extroversion_list)
                    
                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Conscientiousness_list):
                    consci_count +=1
                    consci_ratio = consci_count/len(self.Conscientiousness_list)
                    
                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Neuroticism_list):
                    neuro_count +=1
                    neuro_ratio = neuro_count/len(self.Neuroticism_list)


            max_score = max(open_ratio,agree_ratio,extro_ratio,consci_ratio,neuro_ratio)
            max_list.append(max_score)
            

            if max_score == 0:
                x = "Unknown"
            elif max_score == open_ratio:
                x = "Openness"
            elif max_score == agree_ratio:
                x = "Agreeableness"
            elif max_score == extro_ratio:
                x = "Extroversion"
            elif max_score == consci_ratio:
                x = "Conscientiousness"
            elif max_score == neuro_ratio:
                x = "Neuroticism"


            persona_list.append(x)


        self.df_test_cleaned["personality_score"] = max_list
        self.df_test_cleaned["Personality"] = persona_list
    



In [5]:
class Training_predict_data(CleanData):
   def create_frequency(self):
        freq_d = {}
        for i in range(len(self.df_train_cleaned.index)):
            for word in self.df_train_cleaned["tokenized_tweets"][i]:
                pair = (word, self.df_train_cleaned["sentiment"][i])
                if pair in freq_d:
                    freq_d[pair] += 1
                else:
                    freq_d[pair] = freq_d.get(pair, 1)
    
        return freq_d
    
   def train_naive_bayes(self,freqs, train_x, train_y):
        loglikelihood = {}
        logprior = 0

        unique_words = set([pair[0] for pair in freqs.keys()])

        V = len(unique_words)

        N_pos = N_neg = 0
        for pair in freqs.keys():
            if pair[1]>0:
                N_pos+=freqs[(pair)]

            else:
                N_neg += freqs[(pair)]
        D = train_y.shape[0]

        D_pos = sum(train_y)

        D_neg = D-sum(train_y)

        logprior = np.log(D_pos) - np.log(D_neg)

        for word in unique_words:
            freq_pos = freqs.get((word, 1),0)
            freq_neg = freqs.get((word, 0), 0)
            p_w_pos = (freq_pos+1)/(N_pos+V)
            p_w_neg = (freq_neg+1)/(N_neg+V)
            loglikelihood[word] = np.log(p_w_pos/p_w_neg)
        return logprior, loglikelihood
    
   def naive_bayes_predict(self,logprior,loglikelihood):
       sentiment_score = []
       for i in range(len(self.df_test_cleaned.index)):
               p = 0
               p+=logprior
               for word in self.df_test_cleaned["tokenized_tweets"][i]:
                 if word in loglikelihood:
                   p+=loglikelihood[word]
               sentiment_score.append(p)
       self.df_test_cleaned["sentiment_score"] = sentiment_score
    
   def sentiment_persona(self):
      sentiment_persona = []
      for i in range(len(self.df_test_cleaned.index)):
                score  = self.df_test_cleaned["sentiment_score"][i]
                if (-0.5 < score < 0.5):
                    x  = "Neutral"
                elif (score < -0.5):
                    x = "Negative"
                elif (score > 0.5):
                    x = "Positive"
                sentiment_persona.append(x)
      self.df_test_cleaned["sentiment_persona"] = sentiment_persona

In [6]:
Personality = ['Openness', 'Agreeableness','Conscientiousness' , 'Extroversion' , 'Neuroticism']
# clean_data = CleanData(df)
train_data = Training_predict_data(df_test, df_train)
train_data.read_persona_dict(Personality)
train_data.pre_process("test")
train_data.pre_process("train")
train_data.get_persona_scores()
freq_d=train_data.create_frequency()

In [7]:
logprior, loglikelihood = train_data.train_naive_bayes(freq_d, train_x, train_y)

train_data.naive_bayes_predict(logprior,loglikelihood)

In [8]:
train_data.sentiment_persona()
train_data.df_test_cleaned


,name,text,tokenized_tweets,personality_score,Personality,sentiment_score,sentiment_persona
0,Virat Kohli,The difference between a good attempt and a go...,"[the, difference, good, attempt, good, catch, ...",0.014085,Agreeableness,2.653895,Positive
1,Virat Kohli,I go from hot and uncomfortable to cool and re...,"[i, go, hot, uncomfortable, cool, relaxed, tim...",0.028169,Agreeableness,4.614811,Positive
2,Virat Kohli,Always a pleasure to see you paji. ⭐🐐\r\n@sach...,"[always, pleasure, see, paji, ⭐🐐, sachin_rt]",0.000000,Unknown,2.489529,Positive
3,Virat Kohli,Job done. ✌️\r\n@RCBTweets,"[job, done, ✌️, rcbtweets]",0.000000,Unknown,1.685999,Positive
4,Virat Kohli,Keep the grind on.,"[keep, grind]",0.000000,Unknown,0.914035,Positive
...,...,...,...,...,...,...,...
5795,joliestweet,"I AM READY, ANGIE!!","[i, am, ready, angie]",0.000000,Unknown,-0.162728,Neutral
5796,joliestweet,@ManaByte: The Eternals Trailer is Coming Soon...,"[manabyte, the, eternals, trailer, coming, soon]",0.000000,Unknown,-0.268625,Neutral
5797,joliestweet,@Toastiewiththe: AngelinaJolie Angelina attend...,"[toastiewiththe, angelinajolie, angelina, atte...",0.011364,Extroversion,2.130003,Positive
5798,joliestweet,@ChronicComicsYT: This is by far the only thin...,"[chroniccomicsyt, this, far, thing, i, 'm, tru...",0.014286,Conscientiousness,2.503399,Positive


In [9]:
train_data.df_test_cleaned.groupby(['name','Personality']).count()

text  tokenized_tweets  personality_score  \
name         Personality                                                    
Barack Obama Agreeableness       159               159                159   
             Conscientiousness   167               167                167   
             Extroversion         45                45                 45   
             Neuroticism          23                23                 23   
             Openness             70                70                 70   
             Unknown             736               736                736   
Kanye West   Agreeableness       114               114                114   
             Conscientiousness    57                57                 57   
             Extroversion         41                41                 41   
             Neuroticism          27                27                 27   
             Openness             75                75                 75   
             Unknown            1233              1233               1233   
Virat Kohli  Agreeableness       319               319                319   
             Conscientiousness    90                90                 90   
             Extroversion        104               104                104   
             Neuroticism          18                18                 18   
             Openness             65                65                 65   
             Unknown            1004              1004               1004   
joliestweet  Agreeableness       113               113                113   
             Conscientiousness    39                39                 39   
             Extroversion         74                74                 74   
             Neuroticism          28                28                 28   
             Openness             46                46                 46   
             Unknown            1153              1153               1153   

                                sentiment_score  sentiment_persona  
name         Personality                                            
Barack Obama Agreeableness                  159                159  
             Conscientiousness              167                167  
             Extroversion                    45                 45  
             Neuroticism                     23                 23  
             Openness                        70                 70  
             Unknown                        736                736  
Kanye West   Agreeableness                  114                114  
             Conscientiousness               57                 57  
             Extroversion                    41                 41  
             Neuroticism                     27                 27  
             Openness                        75                 75  
             Unknown                       1233               1233  
Virat Kohli  Agreeableness                  319                319  
             Conscientiousness               90                 90  
             Extroversion                   104                104  
             Neuroticism                     18                 18  
             Openness                        65                 65  
             Unknown                       1004               1004  
joliestweet  Agreeableness                  113                113  
             Conscientiousness               39                 39  
             Extroversion                    74                 74  
             Neuroticism                     28                 28  
             Openness                        46                 46  
             Unknown                       1153               1153

In [10]:
train_data.df_test_cleaned.groupby(['Personality']).count()


,name,text,tokenized_tweets,personality_score,sentiment_score,sentiment_persona
Personality,,,,,,
Agreeableness,705,705,705,705,705,705
Conscientiousness,353,353,353,353,353,353
Extroversion,264,264,264,264,264,264
Neuroticism,96,96,96,96,96,96
Openness,256,256,256,256,256,256
Unknown,4126,4126,4126,4126,4126,4126


In [11]:
train_data.df_test_cleaned.groupby(['name','sentiment_persona']).count()

text  tokenized_tweets  personality_score  \
name         sentiment_persona                                              
Barack Obama Negative            390               390                390   
             Neutral             211               211                211   
             Positive            599               599                599   
Kanye West   Negative            400               400                400   
             Neutral             707               707                707   
             Positive            440               440                440   
Virat Kohli  Negative            284               284                284   
             Neutral             302               302                302   
             Positive           1014              1014               1014   
joliestweet  Negative            442               442                442   
             Neutral             467               467                467   
             Positive            544               544                544   

                                Personality  sentiment_score  
name         sentiment_persona                                
Barack Obama Negative                   390              390  
             Neutral                    211              211  
             Positive                   599              599  
Kanye West   Negative                   400              400  
             Neutral                    707              707  
             Positive                   440              440  
Virat Kohli  Negative                   284              284  
             Neutral                    302              302  
             Positive                  1014             1014  
joliestweet  Negative                   442              442  
             Neutral                    467              467  
             Positive                   544              544

In [12]:
newdf= train_data.df_test_cleaned
df2= newdf.groupby(['name','Personality','sentiment_persona']).size()
print(df2)
df2.index

name          Personality        sentiment_persona
Barack Obama  Agreeableness      Negative              32
                                 Neutral               30
                                 Positive              97
              Conscientiousness  Negative              47
                                 Neutral               32
                                                     ... 
joliestweet   Openness           Neutral               11
                                 Positive              22
              Unknown            Negative             336
                                 Neutral              408
                                 Positive             409
Length: 72, dtype: int64


MultiIndex([('Barack Obama',     'Agreeableness', 'Negative'),
            ('Barack Obama',     'Agreeableness',  'Neutral'),
            ('Barack Obama',     'Agreeableness', 'Positive'),
            ('Barack Obama', 'Conscientiousness', 'Negative'),
            ('Barack Obama', 'Conscientiousness',  'Neutral'),
            ('Barack Obama', 'Conscientiousness', 'Positive'),
            ('Barack Obama',      'Extroversion', 'Negative'),
            ('Barack Obama',      'Extroversion',  'Neutral'),
            ('Barack Obama',      'Extroversion', 'Positive'),
            ('Barack Obama',       'Neuroticism', 'Negative'),
            ('Barack Obama',       'Neuroticism',  'Neutral'),
            ('Barack Obama',       'Neuroticism', 'Positive'),
            ('Barack Obama',          'Openness', 'Negative'),
            ('Barack Obama',          'Openness',  'Neutral'),
            ('Barack Obama',          'Openness', 'Positive'),
            ('Barack Obama',           'Unknown', 'Nega

In [13]:
df3= newdf.groupby(['name','Personality','sentiment_persona']).size().reset_index(name = 'counter')
df3

,name,Personality,sentiment_persona,counter
0,Barack Obama,Agreeableness,Negative,32
1,Barack Obama,Agreeableness,Neutral,30
2,Barack Obama,Agreeableness,Positive,97
3,Barack Obama,Conscientiousness,Negative,47
4,Barack Obama,Conscientiousness,Neutral,32
...,...,...,...,...
67,joliestweet,Openness,Neutral,11
68,joliestweet,Openness,Positive,22
69,joliestweet,Unknown,Negative,336
70,joliestweet,Unknown,Neutral,408


In [14]:
df3.groupby(['name','Personality','sentiment_persona'])

In [15]:
name = df3.name.unique()
print(name)
Person = df3.Personality.unique()
Personality = Person[0:5]
print(Personality)


['Barack Obama' 'Kanye West' 'Virat Kohli' 'joliestweet']
['Agreeableness' 'Conscientiousness' 'Extroversion' 'Neuroticism'
 'Openness']


In [16]:
df2.loc['Virat Kohli','Agreeableness','Neutral']

28

In [17]:
for i in df2.index :
    print(df2.loc[i])
    

32
30
97
47
32
88
9
7
29
12
3
8
16
8
46
274
131
331
34
17
63
20
4
33
14
4
23
17
4
6
27
25
23
288
653
292
28
28
263
13
12
65
15
13
76
7
3
8
12
13
40
209
233
562
48
23
42
10
7
22
19
11
44
16
7
5
13
11
22
336
408
409


In [18]:
for i in name:
    
    print(f"\nPersonality Analysis of {i} is : ")
    for j in Personality:
        if(df2[i,j,'Neutral'] > df2[i,j,'Negative'] and df2[i,j,'Neutral'] > df2[i,j,'Positive'] ) :
            print(f" The {i} is moderately {j}  type")
        elif(df2[i,j,'Negative'] > df2[i,j,'Neutral'] and df2[i,j,'Negative'] > df2[i,j,'Positive'] ) :
            print(f" The {i} is least {j} type ")
        elif(df2[i,j,'Positive'] > df2[i,j,'Neutral'] and df2[i,j,'Positive'] > df2[i,j,'Negative'] ) :
            print(f" The {i} is highly {j} type ")
        else:
            print(f"The {j} type can not be clearly defined")

        


Personality Analysis of Barack Obama is : 
 The Barack Obama is highly Agreeableness type 
 The Barack Obama is highly Conscientiousness type 
 The Barack Obama is highly Extroversion type 
 The Barack Obama is least Neuroticism type 
 The Barack Obama is highly Openness type 

Personality Analysis of Kanye West is : 
 The Kanye West is highly Agreeableness type 
 The Kanye West is highly Conscientiousness type 
 The Kanye West is highly Extroversion type 
 The Kanye West is least Neuroticism type 
 The Kanye West is least Openness type 

Personality Analysis of Virat Kohli is : 
 The Virat Kohli is highly Agreeableness type 
 The Virat Kohli is highly Conscientiousness type 
 The Virat Kohli is highly Extroversion type 
 The Virat Kohli is highly Neuroticism type 
 The Virat Kohli is highly Openness type 

Personality Analysis of joliestweet is : 
 The joliestweet is least Agreeableness type 
 The joliestweet is highly Conscientiousness type 
 The joliestweet is highly Extroversion t